# Thuật toán ID3 cho dữ liệu định tính

In [8]:
import pandas as pd
import numpy as np

feature: đặc trưng sẽ được sử dụng để phân tách các nhánh tiếp theo của cây.
value: giá trị của đặc trưng, được sử dụng để phân tách các nhánh tiếp theo của cây.
label: nhãn của nút, được sử dụng để dự đoán kết quả.
branches: tập hợp các nút con được phân tách từ nút hiện tại.

In [9]:
class Node:
    def __init__(self, feature=None, value=None, label=None, branches=None):
        self.feature = feature
        self.value = value
        self.label = label
        self.branches = branches

Tính entropy:
\begin{aligned}
H(\mathbf{p}) = -\sum_{i=1}^n p_i \log(p_i)\quad\quad (1)
\end{aligned}

In [10]:
def entropy(data):
    label_freq = data.iloc[:, -1].value_counts(normalize=True)
    print(label_freq)
    entropy = 0
    for p in label_freq:
        entropy += -p * np.log2(p)
    return entropy

data = pd.read_csv("datasets/playing.csv")
print(data['Decision'])
entropy(data)


0      no
1      no
2     yes
3     yes
4     yes
5      no
6     yes
7      no
8     yes
9     yes
10    yes
11    yes
12    yes
13     no
Name: Decision, dtype: object
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64


0.9402859586706311

In [11]:
def information_gain(data, feature):

    total_entropy = entropy(data)
    values, counts = np.unique(data[feature], return_counts=True)
    weighted_entropy = 0
    for i in range(len(values)):
        p = counts[i]/np.sum(counts)
        weighted_entropy += p * entropy(data.where(data[feature] == values[i]).dropna())
    information_gain = total_entropy - weighted_entropy
    return information_gain





In [13]:
data = pd.read_csv("datasets/playing.csv")
features = list(data.columns[1:-1])
for i in features:
    gain = information_gain(data,i)
    print(f"{i}: {gain}")


Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
yes    1.0
Name: proportion, dtype: float64
Decision
yes    0.6
no     0.4
Name: proportion, dtype: float64
Decision
no     0.6
yes    0.4
Name: proportion, dtype: float64
outlook: 0.24674981977443933
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
yes    0.75
no     0.25
Name: proportion, dtype: float64
Decision
no     0.5
yes    0.5
Name: proportion, dtype: float64
Decision
yes    0.666667
no     0.333333
Name: proportion, dtype: float64
temperature: 0.02922256565895487
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
no     0.571429
yes    0.428571
Name: proportion, dtype: float64
Decision
yes    0.857143
no     0.142857
Name: proportion, dtype: float64
humidity: 0.15183550136234159
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
no     0.5
yes    0.5
Name: proportion, dtype: float64
Decision
yes   

Các tham số đầu vào của hàm bao gồm:

- data: dữ liệu đầu vào được lưu dưới dạng DataFrame của thư viện Pandas.
- features: danh sách các thuộc tính được sử dụng để xây dựng cây quyết định.
- target: tên của thuộc tính đích, tức là thuộc tính cần được phân loại.
Trong hàm, chúng ta đầu tiên kiểm tra xem liệu tất cả các mẫu đầu vào có cùng nhãn hay không. Nếu có, chúng ta trả về một nút lá với nhãn đó. Nếu không, chúng ta kiểm tra xem đã sử dụng hết các thuộc tính để xây dựng cây chưa. Nếu rồi, chúng ta trả về một nút lá với nhãn được xác định bằng cách chọn nhãn xuất hiện nhiều nhất trong tập dữ liệu.

Nếu chưa, chúng ta chọn thuộc tính có độ lợi thông tin (information gain) lớn nhất để phân chia tập dữ liệu. Sau đó, chúng ta tạo một nút trên cây với thuộc tính được chọn và lặp lại quá trình trên từng giá trị của thuộc tính này. Điều này được thực hiện bằng cách tái đệ quy hàm ID3 với tập dữ liệu con được chọn và danh sách thuộc tính khác (loại bỏ thuộc tính đã chọn) cho đến khi chúng ta đạt được điều kiện dừng.

Cuối cùng, chúng ta trả về cây quyết định đã xây dựng được dưới dạng một đối tượng Node trong Python. Cây quyết định này có thể được sử dụng để phân loại các mẫu mới dựa trên các giá trị của các thuộc tính được chỉ định.

In [14]:
def id3(data, features, target):
    if len(np.unique(data[target])) <= 1:
        return Node(label=data[target].iloc[0])
    elif len(features) == 0:
        mode_target = data[target].mode()[0]
        return Node(label=mode_target)
    else:
        best_gain = 0
        best_feature = ""
        for i in features:
            if information_gain(data, i) > best_gain:
                best_gain = information_gain(data, i)
                best_feature = i
        tree = Node(feature=best_feature)
        for value in np.unique(data[best_feature]): 
            sub_data = data.where(data[best_feature] == value).dropna()
            print(sub_data)
            sub_tree = id3(sub_data, [x for x in features if x != best_feature], target)
            tree.branches = tree.branches or {}
            tree.branches[value] = sub_tree
        return tree


In [21]:
# Để hiển thị cây quyết định được tạo ra bằng thuật toán ID3
def print_tree(node, depth=0):
    if node.label is not None:
        print("         "*depth, "Leaf:", node.label)
    else:
        print("         "*depth, node.feature)
        for value, subtree in node.branches.items():
            print("  "*(depth+1), value)
            print_tree(subtree, depth+2)


In [15]:
def predict(tree, new_data):
    try:
        if tree.label is not None:
            return tree.label
        else:
            value = new_data[tree.feature]
            subtree = tree.branches[value]
            return predict(subtree, new_data)
    except:
        print("Dữ liệu sai!")
    


In [16]:
data = pd.read_csv("datasets/playing.csv")
data


,id,outlook,temperature,humidity,wind,Decision
0,1,sunny,hot,high,weak,no
1,2,sunny,hot,high,strong,no
2,3,overcast,hot,high,weak,yes
3,4,rainy,mild,high,weak,yes
4,5,rainy,cool,normal,weak,yes
5,6,rainy,cool,normal,strong,no
6,7,overcast,cool,normal,strong,yes
7,8,sunny,mild,high,weak,no
8,9,sunny,cool,normal,weak,yes
9,10,rainy,mild,normal,weak,yes


In [17]:
target = "Decision"
features = list(data.columns[1:-1])
tree = id3(data, features, target)
new_data = {"outlook": "rainy", "temperature": "hot",
            "humidity": "normal", "windy": "weak"}
prediction = predict(tree, new_data)
print(f"Kết quả dự đoán: {prediction}")


Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
yes    1.0
Name: proportion, dtype: float64
Decision
yes    0.6
no     0.4
Name: proportion, dtype: float64
Decision
no     0.6
yes    0.4
Name: proportion, dtype: float64
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
yes    1.0
Name: proportion, dtype: float64
Decision
yes    0.6
no     0.4
Name: proportion, dtype: float64
Decision
no     0.6
yes    0.4
Name: proportion, dtype: float64
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
yes    0.75
no     0.25
Name: proportion, dtype: float64
Decision
no     0.5
yes    0.5
Name: proportion, dtype: float64
Decision
yes    0.666667
no     0.333333
Name: proportion, dtype: float64
Decision
yes    0.642857
no     0.357143
Name: proportion, dtype: float64
Decision
no     0.571429
yes    0.428571
Name: proportion, dtype: float64
Decision
yes    0.857143
no     0.142857
Name: proportion, dtype

In [22]:
print_tree(tree)


 outlook
   overcast
                   Leaf: yes
   rainy
                   wind
       strong
                                     Leaf: no
       weak
                                     Leaf: yes
   sunny
                   humidity
       high
                                     Leaf: no
       normal
                                     Leaf: yes


# Decision Tree trong `sklearn`

Scikit-Learn sử dụng thuật toán `CART` chỉ tạo ra các cây nhị phân: các nút không là lá luôn có hai nút con (nghĩa là các câu hỏi chỉ có câu trả lời có/không).

Thuật toán này sử dụng một thuộc tính `k` trong tập các tập thuộc tính và ngưỡng $t_k$ là ngưỡng để phân chia dữ liệu thành 2 phần

## Gini Impurity (Gini index) 

- là một phương pháp đo độ tinh khiết của một tập hợp dữ liệu được sử dụng trong các thuật toán học máy

- Gini index đo lường khả năng chọn một mẫu ngẫu nhiên từ tập hợp dữ liệu và xác định xem nó sẽ được phân loại đúng trong tập hợp đó hoặc không

- Mục tiêu của việc sử dụng Gini index trong CART là tìm thuộc tính và giá trị ngưỡng phân chia mà cho kết quả tối ưu trong việc phân loại các mẫu vào các nhóm khác nhau.

$$\text{Gini Impurity} = \sum_{i=1}^{K}p_{i}(1-p_{i}) = 1 - \sum_{i=1}^{K}p_{i}^2$$

- Tính bằng cách tính tổng xác suất lỗi phân loại khi một quan sát được phân loại ngẫu nhiên theo phân phối xác suất của các nhãn.

## CART Hàm mất mát cho bài toán phân loại

$$J(k, t_k) = \frac{m_{\text{left}}}{m} G_{\text{left}} + \frac{m_{\text{right}}}{m} G_{\text{right}}$$

$$
\begin{cases}
    G_{\text{left/right}} \text{ measures the impurity of the left/right subset} \\
    m_{\text{left/right}} \text{ is the number of instances in the left/right subset}
\end{cases}
$$

Có thể chọn phép đo thành Entropy Impurity bằng cách đặt `criterion` bằng `entropy` trong `sklearn`

Việc sử dụng Gini Impurity hay Entropy làm tiêu chí phân tách tùy thuộc vào vấn đề và dữ liệu. Dưới đây là một số cân nhắc chung: 

- Gini Impurity có xu hướng tính toán nhanh hơn entropy, điều này làm cho nó trở thành một lựa chọn tốt khi xử lý các tập dữ liệu lớn. 

- Gini Impurity là một tiêu chí tốt để sử dụng khi các lớp cân bằng tốt, trong khi entropy có thể hoạt động tốt hơn khi các lớp không cân bằng. 

- Entropy là một tiêu chí cung cấp nhiều thông tin hơn so với Gini Impurity, vì nó tính đến xác suất của từng loại, thay vì chỉ các tần số. Nó có thể dẫn đến một cây cân bằng và chính xác hơn trong một số trường hợp. 

- Cả Gini Impurity và entropy đều được sử dụng rộng rãi và đã được chứng minh là hoạt động tốt trong nhiều ứng dụng trong thế giới thực. 


## CART Hàm mất mát cho bài toán hồi quy

Vẫn là cách hoạt động cũ, nhưng thay vì sư dụng `Gini Impurity` thì có thể huấn luyện mô hình với `MSE` (Mean Square Error), làm cho `MSE` là nhỏ nhất:

$$J(k, t_k) = \frac{m_{\text{left}}}{m} MSE_{\text{left}} + \frac{m_{\text{right}}}{m} MSE_{\text{right}}$$

$$
\begin{cases}
    MSE_{\text{node}} = \sum_{i \in \text{node}} (\hat{y}_{\text{node}} - y^{(i)})^2 \text{ measures the impurity of the left/right subset} \\
    \hat{y}_{\text{node}} = \frac{1}{m_{\text{node}}} \sum_{i \in \text{node}} y^{(i)} \\
    m_{\text{left/right}} \text{ is the number of instances in the left/right subset}
\end{cases}
$$